In [20]:
import time
from dotenv import load_dotenv
import requests
import os
from datetime import datetime,timedelta
import calendar
from datetime import date

# выгрузка токена из .env
load_dotenv(dotenv_path='utils/.env')
wb_token = os.getenv('wb_token')
wb_token_test = os.getenv('wb_token_test')

In [25]:
# подготовка к запросам, установка headers + функция разбивки
headers = {
    "Authorization": wb_token_test
}

def split_period(start_str, end_str, format_str='%Y-%m-%d', max_days=31):
    start = datetime.strptime(start_str, format_str).date()
    end = datetime.strptime(end_str, format_str).date()
    if start > end:
        raise ValueError("Начальная дата должна быть раньше или равна конечной")
    intervals = []
    current = start
    while current <= end:
        next_date = min(current + timedelta(days=max_days - 1), end)
        intervals.append((current, next_date))
        current = next_date + timedelta(days=1)
    return intervals

In [22]:
# Financial Reports / Финансовые отчеты
url = "https://statistics-api-sandbox.wildberries.ru" + "/api/v5/supplier/reportDetailByPeriod"
params = {
    "dateFrom": "2025-10-25",
    "dateTo": "2025-10-29"
}
res = requests.get(url, params=params, headers=headers)
res.json()
data_to_output = {}
data_to_output['wb_art'] = res.json()[0].get('nm_id')
data_to_output['sa_art'] = res.json()[0].get('sa_name')
data_to_output['realizationreport_id'] = res.json()[0].get('realizationreport_id')
data_to_output['brand_name'] = res.json()[0].get('brand_name')
res.json()

KeyError: 0

In [18]:
# Баланс продавца
url = base_url + "/api/v1/account/balance"
res = requests.get(url, headers=headers)
print(res.json())

In [78]:
# список документов
url = base_url + "/api/v1/documents/list"
res = requests.get(url, headers=headers)
res.json()

{'data': {'documents': [{'serviceName': 'upd-151953033',
    'name': 'УПД №151953033 от 05.11.2023',
    'category': 'Universal transfer document',
    'extensions': ['zip'],
    'creationTime': '2023-11-08T08:01:38.897Z',
    'viewed': True},
   {'serviceName': 'actprofit-151953033',
    'name': 'Акт взаимозачета №151953033 от 05.11.2023',
    'category': 'Offsetting act',
    'extensions': ['zip'],
    'creationTime': '2023-11-08T05:22:38.789Z',
    'viewed': True},
   {'serviceName': 'weekly-implementation-report-47097567',
    'name': 'Отчёт по оферте №47097567 от 30.10.2023',
    'category': 'Weekly implementation report',
    'extensions': ['zip'],
    'creationTime': '2023-11-07T12:33:39.934Z',
    'viewed': True},
   {'serviceName': 'weekly-implementation-report-45969165',
    'name': 'Отчёт по оферте №45969165 от 09.10.2023',
    'category': 'Weekly implementation report',
    'extensions': ['zip'],
    'creationTime': '2023-10-17T10:51:15.411Z',
    'viewed': True},
   {'serv

In [82]:
# получение документа
url = base_url + "/api/v1/documents/download"
params = {
    "serviceName": "upd-122335340",
    "extension": ""
}
res = requests.get(url, params=params, headers=headers)
res.json()

{'title': 'Not Found',
 'status': 400,
 'detail': 'Не удалось скачать файл, файл не найден',
 'requestId': 'fa4c3329-7a04-429d-ad55-0d1d7c45a034',
 'origin': 'docs-public-api'}

In [105]:
# воронка продаж
url = "https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail"
params = {
    "period": {"begin": "2023-06-01 20:05:32",
               "end": "2024-03-01 20:05:32"
               },
    "page": "1"
}
res = requests.post(url, json=params, headers=headers)
res.text

'{"data":null,"error":true,"errorText":"Некорректное тело запроса","additionalErrors":[{"field":"request body: period.begin","description":"invalid value 2023-06-01 20:05:32 +0000 UTC, select a date no earlier than 2024-10-29 13:31:27"},{"field":"request body: period.end","description":"invalid value 2024-03-01 20:05:32 +0000 UTC, select a date no earlier than 2024-10-29 13:31:27"}]}\n'

In [14]:
# основные отчеты продажи общие поле: "продажи"
# ограничения: запрос раз в минуту
url = "https://statistics-api.wildberries.ru/api/v1/supplier/sales"
test_url = "https://statistics-api-sandbox.wildberries.ru/api/v1/supplier/sales"
dateFrom = "2022-10-29T13:40:59"
params = {
    "dateFrom": dateFrom
}
response_status_429 = True
while response_status_429:
    res = requests.get(test_url, params=params, headers=headers)
    print(f"Статус ответа по {test_url}: {res.status_code}")
    if res.status_code != 200:
        time.sleep(30)
    else:
        response_status_429 = False
data_to_output = res.json()
summ_sells_output = 0
for nm in data_to_output:
    summ_sells_output += nm.get('priceWithDisc')
summ_sells_output

Статус ответа по https://statistics-api-sandbox.wildberries.ru/api/v1/supplier/sales: 200


217742.55999999994

In [68]:
# основные отчеты продажи сделать перегруженную функцию поле: "продажи" и "возвраты"
# ограничения: запрос раз в минуту
url = "https://statistics-api.wildberries.ru/api/v1/supplier/sales"
test_url = "https://statistics-api-sandbox.wildberries.ru/api/v1/supplier/sales"
dateFrom = "2022-10-29T13:40:59"
params = {
    "dateFrom": dateFrom
}
response_status_429 = True
while response_status_429:
    res = requests.get(test_url, params=params, headers=headers)
    print(f"Статус ответа по {test_url}: {res.status_code}")
    if res.status_code != 200:
        time.sleep(5)
    else:
        response_status_429 = False
data_to_output = res.json()
summ_sells_output = 0
summ_return_output = 0
for nm in data_to_output:
    if str(nm.get("saleID")).lower().strip().find("r") != -1:
        summ_return_output += nm.get('priceWithDisc')
    summ_sells_output += nm.get('priceWithDisc')

res.json()

Статус ответа по https://statistics-api-sandbox.wildberries.ru/api/v1/supplier/sales: 200


[{'date': '2025-06-28T15:51:42Z',
  'lastChangeDate': '2025-06-29T10:04:42Z',
  'warehouseName': 'Коледино',
  'warehouseType': 'Склад WB',
  'countryName': 'Россия',
  'oblastOkrugName': 'Центральный федеральный округ',
  'regionName': 'Рославльский',
  'supplierArticle': 'BGC03148',
  'nmId': 324836867,
  'barcode': 'BGC03148',
  'category': 'Телевизоры и аудиотехника',
  'subject': 'Наушники',
  'brand': 'wildberries',
  'techSize': '0',
  'incomeID': 21328641,
  'isSupply': False,
  'isRealization': True,
  'totalPrice': 1835,
  'discountPercent': 65,
  'spp': 0,
  'forPay': 393.63,
  'finishedPrice': 398,
  'priceWithDisc': 474.25,
  'saleID': 'S3207157749',
  'sticker': '',
  'gNumber': '1536700528478129741',
  'srid': '55361630070647446.2.2'},
 {'date': '2025-06-29T15:51:42Z',
  'lastChangeDate': '2025-06-30T10:04:42Z',
  'warehouseName': 'Коледино',
  'warehouseType': 'Склад WB',
  'countryName': 'Россия',
  'oblastOkrugName': '',
  'regionName': '',
  'supplierArticle': 'BGC02

In [129]:
# основные отчеты продажи сделать перегруженную функцию поле: "заказы" и "отмены"
# ограничения: запрос раз в минуту
url = "https://statistics-api.wildberries.ru/api/v1/supplier/orders"
test_url = "https://statistics-api-sandbox.wildberries.ru/api/v1/supplier/orders"
dateFrom = "2022-10-29T13:40:59"
params = {
    "dateFrom": dateFrom
}
response_status_429 = True
while response_status_429:
    res = requests.get(test_url, params=params, headers=headers)
    print(f"Статус ответа по {test_url}: {res.status_code}")
    if res.status_code != 200:
        time.sleep(30)
    else:
        response_status_429 = False
data_to_output = res.json()
summ_sells_orders_output = 0
summ_sells_CancelOrders_output = 0
for nm in data_to_output:
    if not nm.get('isCancel'):
        summ_sells_orders_output += nm.get('priceWithDisc')
    else:
        summ_sells_CancelOrders_output += nm.get('priceWithDisc')
summ_sells_CancelOrders_output



Статус ответа по https://statistics-api-sandbox.wildberries.ru/api/v1/supplier/orders: 401


KeyboardInterrupt: 

In [65]:
# Создание отчета о платном хранении
# ограничения: запрос раз в минуту
url = "https://seller-analytics-api.wildberries.ru/api/v1/paid_storage"
dateFrom = "2023-07-17"
dateTo = "2023-07-23"
params = {
    "dateFrom": dateFrom,
    "dateTo": dateTo
}
print(f"Создание отчета")
response_status_429 = True
while response_status_429:
    res = requests.get(url, params=params, headers=headers)
    print(f"Статус ответа по {url}: {res.status_code}")
    print(f"Ответ сервера по {url}: {res.text}")
    if res.status_code != 200:
        time.sleep(5)
    else:
        response_status_429 = False

id_report = res.json().get('data').get('taskId')

# Получение отчета о платном хранении
# ограничения: запрос раз в минуту
print(f"Получение отчета")
url = f"https://seller-analytics-api.wildberries.ru/api/v1/paid_storage/tasks/{id_report}/download"
params = {
    "task_id": id_report
}
time.sleep(2)
response_status_429 = True
while response_status_429:
    res = requests.get(url, params=params, headers=headers)
    print(f"Статус ответа по {url}: {res.status_code}")
    print(f"Ответ сервера по {url}: {res.text}")
    if res.status_code != 200:
        time.sleep(5)
    else:
        response_status_429 = False
res.json()



Создание отчета
Статус ответа по https://seller-analytics-api.wildberries.ru/api/v1/paid_storage: 200
Ответ сервера по https://seller-analytics-api.wildberries.ru/api/v1/paid_storage: {"data":{"taskId":"a3cf8357-6ff3-41a5-8a43-a47d1b72b11e"}}

Получение отчета
Статус ответа по https://seller-analytics-api.wildberries.ru/api/v1/paid_storage/tasks/a3cf8357-6ff3-41a5-8a43-a47d1b72b11e/download: 429
Ответ сервера по https://seller-analytics-api.wildberries.ru/api/v1/paid_storage/tasks/a3cf8357-6ff3-41a5-8a43-a47d1b72b11e/download: {
    "title": "too many requests",
    "detail": "Limited by global limiter, per seller d15b1872-6dc7-425e-9f8b-40fe3a24e863; See https://dev.wildberries.ru/openapi/api-information",
    "code": "461a0b83d6bd 2950e93b5fda",
    "requestId": "af5f622d80456c50e7d0d86733a6ece2",
    "origin": "s2sauth-ca",
    "status": 429,
    "statusText": "Too Many Requests",
    "timestamp": "2025-10-29T12:40:11Z"
}

Статус ответа по https://seller-analytics-api.wildberries.ru

[]

In [28]:
# Пока основной вариант
# Financial Reports / Финансовые отчеты "продажи" , "возвраты" , "коммисия"
url = "https://statistics-api-sandbox.wildberries.ru" + "/api/v5/supplier/reportDetailByPeriod"
DATE_FORMAT = "%Y-%m-%dT%H:%M:%S.%f"
now_time = datetime.now()
formatted_date = now_time.strftime(DATE_FORMAT)
params = {
    "dateFrom": "2025-08-29",
    "dateTo": "2025-08-29"
}
res = requests.get(url, params=params, headers=headers)
data = res.json()
return_output_sum = 0
sell_all_sum = 0
commission_output = 0
for nm in data:
    return_output_sum += abs(nm.get('return_amount')) * abs(nm.get('retail_price_withdisc_rub'))
    sell_all_sum += abs(nm.get('retail_amount')) * abs(nm.get('retail_price_withdisc_rub'))
    commission_output += nm.get('ppvz_sales_commission')

sell_output_sum = sell_all_sum - return_output_sum
res.json()
sell_all_sum

0

In [7]:
# Получение поля рекламные расходы

# Компании. Информация о компаниях основной https://advert-api.wildberries.ru/adv/v1/promotion/count песочница https://advert-api-sandbox.wildberries.ru/adv/v1/promotion/count
url = "https://advert-api.wildberries.ru/adv/v1/promotion/count"

res = requests.get(url, headers=headers)
data = res.json()
list_company_id = []
if data.get('adverts'):
    for company_status_list in data.get('adverts'):
        for company in company_status_list.get('advert_list'):
            list_company_id.append(company.get('advertId'))

list_company_id



[7104506,
 7898972,
 7900481,
 7025363,
 7025374,
 7025382,
 7025390,
 7042403,
 7348381,
 7857465,
 8052954]

In [8]:
# Статистика. Статистика кампаний поля: "рекламные расходы", ДРР
dateFrom = "2023-06-01"
dateTo = "2023-08-31"

url = "https://advert-api.wildberries.ru/adv/v3/fullstats"
list_company_id_for_post = list_company_id
list_company_id_for_post = [list_company_id[i:i + 100] for i in range(0, len(list_company_id), 100)]
range_date = split_period(dateFrom,dateTo)

data =[]
for period in range_date:
    for company_100 in list_company_id_for_post:
        response_status_429 = True
        params = {
            "ids": ",".join(map(str, list_company_id)),
            "beginDate": period[0],
            "endDate": period[1]
        }
        while response_status_429:
            res = requests.get(url, params=params, headers=headers)
            if res.status_code != 429:
                print(f"Ответ сервера по url:{url} {res.status_code}")
                response_status_429 = False
                data.append(res.json())
            else:
                print(f"Ответ сервера по url:{url} {res.status_code}")
                time.sleep(20)



Ответ сервера по url:https://advert-api.wildberries.ru/adv/v3/fullstats 200
Ответ сервера по url:https://advert-api.wildberries.ru/adv/v3/fullstats 429
Ответ сервера по url:https://advert-api.wildberries.ru/adv/v3/fullstats 200
Ответ сервера по url:https://advert-api.wildberries.ru/adv/v3/fullstats 429
Ответ сервера по url:https://advert-api.wildberries.ru/adv/v3/fullstats 200


In [18]:
new_items_for_load = []
for response_item in data:
    for item in response_item:
        for day_items in item.get('days'):
            new_items = day_items
            new_items['advertid'] = item.get('advertId')
            new_items_for_load.append(new_items)

new_items_for_load


[{'apps': [{'appType': 64,
    'atbs': 0,
    'canceled': 0,
    'clicks': 0,
    'cpc': 0,
    'cr': 0,
    'ctr': 0,
    'nms': [{'atbs': 0,
      'canceled': 0,
      'clicks': 0,
      'cpc': 0,
      'cr': 0,
      'ctr': 0,
      'name': 'Видеоняня с монитором',
      'nmId': 162909138,
      'orders': 0,
      'shks': 0,
      'sum': 4.1,
      'sum_price': 0,
      'views': 27}],
    'orders': 0,
    'shks': 0,
    'sum': 4.1,
    'sum_price': 0,
    'views': 27},
   {'appType': 1,
    'atbs': 0,
    'canceled': 0,
    'clicks': 0,
    'cpc': 0,
    'cr': 0,
    'ctr': 0,
    'nms': [{'atbs': 0,
      'canceled': 0,
      'clicks': 0,
      'cpc': 0,
      'cr': 0,
      'ctr': 0,
      'name': 'Видеоняня с монитором',
      'nmId': 162909138,
      'orders': 0,
      'shks': 0,
      'sum': 1.98,
      'sum_price': 0,
      'views': 13}],
    'orders': 0,
    'shks': 0,
    'sum': 1.98,
    'sum_price': 0,
    'views': 13},
   {'appType': 32,
    'atbs': 1,
    'canceled': 0,


In [156]:
# считаем только нужные артикулы.

list_wb_art = [162909138]
target_sum_output = 0
if list_wb_art: # если есть список артикулов, то
    for answer_period in data:
        if type(answer_period) == list:
            for answer_company in answer_period:
                for days_in_company in answer_company.get('days'):
                    for app_in_day in days_in_company.get('apps'):
                        if app_in_day.get('nms')[0].get('nmId') in list_wb_art:
                            target_sum_output+=app_in_day.get('sum')
else:# если нет списка артикулов, то
      for answer_period in data:
          if type(answer_period) == list:
              for answer_company in answer_period:
                  target_sum_output+=answer_company.get("sum")
target_sum_output

101929.8

In [135]:
summ_sells_orders_output = 10000 # для теста
DRR_output = target_sum_output / summ_sells_orders_output
round(DRR_output)

10